## Byte Pair Encoding

In [7]:
# !pip install tiktoken

In [2]:
import importlib
import importlib.metadata
import tiktoken

print("Tiktoken version: ", importlib.metadata.version("tiktoken"))

Tiktoken version:  0.8.0


In [3]:
tokenizer = tiktoken.get_encoding("gpt2")

In [5]:
text =(
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [6]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


#### Creating input-target pairs

In [10]:
with open("the-verdict.txt", "r") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [11]:
enc_sample = enc_text[50:]

In [14]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [17]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(f"{context}: -----> {desired}")

[290]: -----> 4920
[290, 4920]: -----> 2241
[290, 4920, 2241]: -----> 287
[290, 4920, 2241, 287]: -----> 257


The list to the left of the arrow refers to the input tokens and right of the arrow refers to the target

Repeating the above step with decoded text results as follows:

In [18]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(f"{tokenizer.decode(context)}: -----> {tokenizer.decode([desired])}")

 and: ----->  established
 and established: ----->  himself
 and established himself: ----->  in
 and established himself in: ----->  a


### Implementation of a Data Loader with PyTorch

<div class="alert alert-block alert-warning">
<h4>Steps</h4>
1. Tokenize the text </br>
2. Use a sliding window of context size to chunk the input into overlapping sequences </br>
3. Return the total number of rows in the dataset </br>
4. Return a single row from the dataset 
</div>

In [28]:
#!pip install torch

In [29]:
#!pip install torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [30]:
#import torch
#print(torch.__version__)
#print(torch.cuda.is_available())

In [31]:
from torch.utils.data import Dataset, DataLoader

In [32]:
class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i: i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


<div class="alert alert-block alert-success">
<h4>Creating the DataLoader</h4>
1. Initialize the tokenizer </br>
2. Create the dataset </br>
3. drop_last=True drops the last batch if it is shorter thanthe specified batch_size to prevent loss spikes during training </br>
4. The number of CPU processes to use for preprocessing
</div>

In [33]:
def create_dataloader_v1(text, # dataset provided
                         batch_size=4, # number of cpu/gpu threads to run in parallel
                         max_length=256, # context length
                         stride=128, # overlap between contexts
                         shuffle=True, # shuffle the dataset
                         drop_last=True, # drop the last batch to prevent loss spike
                         num_workers=0): # number of cpu/gpu threads available
    tokenizer = tiktoken.get_encoding("gpt2") # use tiktoken to get the tokenizer

    dataset = GPTDatasetV1(text, tokenizer, max_length, stride) # create dataset by creating torch tensor chunks

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, 
                            drop_last=drop_last, num_workers=num_workers) # create torch dataloader
    
    return dataloader

Testing the dataset and dataloader with batch_size=1 and context_size=4

In [38]:
print(len(raw_text))

20479


In [39]:
import torch
print("Pytorch version: ", torch.__version__)

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

Pytorch version:  2.5.1+cu124
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [40]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs: \n", inputs)
print("Targets: \n", targets)

Inputs: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Targets: 
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


### Vector Embeddings

<div class="alert alert-success">
Random numbers and One Hot Encoding fails to capture the semantic meaning of the token they represent. Without the semantic meaning, the relationship between words (Semantic relationship) is lost.</br>
We have to train a neural network to create vector embeddings. Creating vector embeddings is not easy and is an expensive task.
</div>

In [42]:
#!pip install gensim

In [43]:
import gensim.downloader as api
model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [45]:
word_vectors = model

print(word_vectors["king"].shape)

(300,)


In [48]:
# Example using most similar
print(word_vectors.most_similar(positive=["king", "woman"], negative="man", topn=3))

[('queen', 0.7118193507194519), ('monarch', 0.6189674735069275), ('princess', 0.5902431011199951)]


#### Creating Token Embeddings

##### Testing with 4 words with a vocab size of 6 and 3 dimensions

In [52]:
input_ids = torch.tensor([2,3,5,1])

In [53]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [54]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [55]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)
